In [1]:
#installation
!pip install python-docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 40.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184491 sha256=1a78f6efad37e8df5bb93bdeb2c48172dd3ce99915ff527fe81f28c83e56eb1c
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx


In [2]:
#Bibliothèques
from google.colab import files
import docx
import re

In [3]:
#Fonctions

#1ere Fonction pour convertir un document en texte:
def convert_docx_to_text(docx_file):
    doc = docx.Document(docx_file)
    text = ' \n'.join([paragraph.text for paragraph in doc.paragraphs])
    return text

#2eme Fonction  trouver chapitres qui prend une chaîne de texte en entrée. Il utilise un modèle d'expression régulière pour trouver des titres de chapitre dans le texte.
def trouver_chapitres(text):
    chapitres = []
    pattern = r'^(?:(?:\d+\.\s*|[A-Z]+\.\s*|I+\s+|V+\s+|X+\s+|\bThe\s+|\bChapter\b|\bIntroduction\b|\bConclusion\b|\bTitre\b).*?)$'
    matches = re.findall(pattern, text, re.MULTILINE)
    for match in matches:
        chapitres.append(match)
    return chapitres

def detecter_titres(fichier_word):
    document = docx.Document(fichier_word)
    titres = []

    for paragraph in document.paragraphs:
        if paragraph.style.name.startswith('Heading'):
            titres.append(paragraph.text)

    return titres
#3eme Fonction prend une liste de paragraphes (paragraphes) et une liste de phrases (Titres) en entrée.
    #Il itère sur chaque phrase et paragraphe, et si une phrase est trouvée dans un paragraphe,
    #il supprime la phrase du paragraphe en la remplaçant par une chaîne vide.
def remove_sentences_from_paragraphs(paragraphs, sentences):
    # Itérer sur les phrases
    for sentence in sentences:
        # Itérer sur les paragraphes
        for i, paragraph in enumerate(paragraphs):
            # Vérifier si la phrase existe dans le paragraphe
            if sentence in paragraph:
                # Supprimer la phrase du paragraphe
                paragraphs[i] = paragraph.replace(sentence, '')
    return paragraphs

#4eme Fonction extrait les paragraphes qui apparaissent après le mot de début spécifié dans un fichier DOC et les imprime
def extract_paragraphs_after_start_word(docx_path, start_word):
    doc = docx.Document(docx_path)
    paragraphs = []
    is_start_found = False

    for paragraph in doc.paragraphs:
        if is_start_found:
            paragraphs.append(paragraph.text)
        elif paragraph.text.strip().startswith(start_word):
            paragraphs.append(paragraph.text)
            is_start_found = True

    return paragraphs

#6eme Fonction Enregistrer le texte dans un document Word
def get_user_input():
    text = input("Enter the beginning of text:")
    return text

def save_text_to_docx(text,text2, docx_path):
    doc = docx.Document()

    # Iterate over chapters in text2
    for i, chapter in enumerate(text2, 1):
        # Add chapter number and content
        doc.add_paragraph(f'Titres {i}: {chapter}')

    for i, paragraph in enumerate(text, 1):
        # Add paragraph number and content
        doc.add_paragraph(f'Paragraphe {i}: {paragraph}')

    doc.save(docx_path)

In [4]:
uploaded_files = files.upload()
# Ouvrir la boîte de dialogue pour choisir le fichier
for file_name, file_content in uploaded_files.items():
    print("File uploaded:", file_name)
    with open(file_name, 'wb') as f:
        f.write(file_content)

Saving IBO - Paragraphe - 03.docx to IBO - Paragraphe - 03.docx
File uploaded: IBO - Paragraphe - 03.docx


In [5]:
# Ouvrir le fichier Word et afficher son contenu
doc = docx.Document(file_name)
text = convert_docx_to_text(file_name)

In [6]:
#extrait les chapitres
titres_detectes = detecter_titres(file_name)
#extrait les Paragraphes
intro_paragraphs = extract_paragraphs_after_start_word(file_name, 'DEBUT_ANALYSE_DELYS')
#supprimer les titres dans les paragraphes
updated_paragraphs1 = remove_sentences_from_paragraphs(intro_paragraphs, titres_detectes)

In [7]:
print('Voici Les Paragraphes : ')
paragraphs_final=[]
for i, paragraph in enumerate(updated_paragraphs1, 1):
  lines = paragraph.split('\n')
  if paragraph.strip() != ''    :
    print('Paragraph', i, ':', paragraph)
    paragraphs_final.append(paragraph)
    print()

Voici Les Paragraphes : 
Paragraph 1 : DEBUT_ANALYSE_DELYS Le dossier de spécifications externes regroupe l’ensemble des documents décrivant les formats d’échange avec le portail public de dans le cadre de la généralisation de la électronique entre assujettis à la TVA, telle qu’elle résulte  de l’article 195 de la loi de finances pour 2021 et de l’ordonnance n°2021-1190 du 15 septembre 2021 relative à la généralisation de la électronique dans les transactions entre assujettis à la taxe sur la valeur ajoutée et à la transmission des données de transaction. 

Paragraph 4 : Ce document s’organise en plusieurs parties afin de définir le contexte et objectifs de la électronique, son cadre réglementaire, la description fonctionnelle de la solution et des formats portant sur les flux d’échange. L’annuaire, les cas de et les protocoles de raccordement sont également détaillés dans ce document.  

Paragraph 6 : Les spécifications externes entrent dans le cadre de l’organisation, du développemen

In [8]:
# ajouter des paragraphes avec le texte souhaité et enregistrez le document.
docx_file = "output_document.docx"
save_text_to_docx(paragraphs_final,titres_detectes, docx_file)

In [9]:
def create_title_document(document_path, output_path):
    doc = docx.Document(document_path)
    title_doc = docx.Document()

    nb_phrases = 1
    debut_parag = False
    fin_parag = False
    nb_para = 1

    for paragraph in doc.paragraphs:
        sentence = paragraph.text.strip()

        if sentence:
            if sentence[0].isdigit():
                nb_phrases = 0
                debut_parag = True
                fin_parag = True
            else:
                nb_phrases += 1
                if debut_parag:
                    nb_para += 1
                    title_doc.add_paragraph("-------------> DEBUT PARAGRAPHE NUM " + str(nb_para))
                    title_doc.add_paragraph(sentence)
                    debut_parag = False
                else:
                    title_doc.add_paragraph(sentence)


    title_doc.save(output_path)
    print("Le document des titres et des lignes à traiter a été créé avec succès.")

In [11]:
create_title_document(file_name, 'output_path.docx')

Le document des titres et des lignes à traiter a été créé avec succès.
